In [1]:
from tensorflow import keras
import sklearn
import IPython
import speech_recognition as sr
from pydub import AudioSegment
import subprocess
import os
import csv
import numpy as np
import pandas as pd
from tqdm import tqdm
from pydub.silence import detect_nonsilent
import librosa
import wave
import contextlib
from konlpy.tag import Kkma
import pandas as pd
import math
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from typing import List
from lexrankr import LexRank
import cx_Oracle
from moviepy.editor import *
from pydub import AudioSegment
import os

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
word_list = pd.read_csv("./불용어.csv" , encoding="utf-8-sig")

In [3]:
bul_list = []
for bul in word_list["불용어"] :
    bul_list.append(bul)

In [4]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
#         self.okt = Okt()
#         self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
#         ,"아", "휴", "아이구","다른" ,"얘기","아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]
        self.stopwords = bul_list
    def url2sentences(self, text):
#     article = Article(url, language='ko')
#     article.download()
#     article.parse()
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.kkma.nouns(str(sentence))
                    if noun not in self.stopwords and len(noun) > 1]))
        return nouns

In [5]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}


In [6]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

    


In [7]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)
            
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
            
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords
    

In [8]:
#영상분석 insert
def insert_videos(t):
    conn = cx_Oracle.connect("hr/hr@222.102.104.172:1521/XE")
    cursor=conn.cursor()
    sql = "insert into videos values(videos_seq.nextval,:1,:2,:3,:4,default,null,:5)"
    cursor.execute(sql,t)
    cursor.close()
    conn.commit()
    conn.close()

In [9]:
def insert_video_analyses(t):
    conn = cx_Oracle.connect("hr/hr@222.102.104.172:1521/XE")
    cursor=conn.cursor()
    sql = "insert into video_analyses values(video_analyses_seq.nextval,:1,:2,:3,:4,:5,null)"
    cursor.execute(sql,t)
    cursor.close()
    conn.commit()
    conn.close()

In [10]:
def insert_keywords(t):
    conn = cx_Oracle.connect("hr/hr@222.102.104.172:1521/XE")
    cursor=conn.cursor()
    sql = "insert into keywords values(keywords_seq.nextval,:1,:2,:3,:4)"
    cursor.execute(sql,t)
    cursor.close()
    conn.commit()
    conn.close()

In [11]:
def select_videos_seq() :
    conn = cx_Oracle.connect("hr/hr@222.102.104.172:1521/XE")
    cursor=conn.cursor()
    sql = "SELECT LAST_NUMBER FROM USER_SEQUENCES WHERE SEQUENCE_NAME = 'VIDEOS_SEQ'"
    cursor.execute(sql,)
    result=cursor.fetchall()[0][0]-20 
    cursor.close()
    conn.close()
    return result
    

In [12]:
from typing import List
from konlpy.tag import Okt

class OktTokenizer:
    okt: Okt = Okt()

    def __call__(self, text: str) -> List[str]:
        tokens: List[str] = self.okt.pos(text, norm=True, stem=True, join=True)
        return tokens

In [13]:
kkm=Kkma()

In [14]:
import os
print(os.getcwd())

C:\Users\SMHRD\3차 프로젝트


In [15]:
import os
 
path_dir = './data/공무원/'
 
file_list = os.listdir(path_dir)
file_list

['new공무원_국어.mp3',
 'new공무원_국어.wav',
 '공무원_경제학.wav',
 '공무원_국어.mp3',
 '공무원_국어.mp4',
 '공무원_국어.wav',
 '공무원_한국사.wav',
 '공무원_행정법.wav',
 '공무원_회계학.wav']

In [16]:
for file_name in tqdm(file_list[1:2]) :
    with contextlib.closing(wave.open(path_dir+file_name,'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
        print(duration)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 201.67it/s]

1493.76


In [17]:
pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))

In [18]:
model=keras.models.load_model('my_h5_model.h5')

In [19]:
def split_wav(data, sample_rate, start, end):
    start *= sample_rate
    end *= sample_rate
    return data[int(start):int(end)]

In [20]:
def cal_duration(file_path) :
    with contextlib.closing(wave.open(file_path,'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
    return duration

In [21]:
def cut_time(file_path) :
    y, sr = librosa.load(file_path, sr=16000)

    np_padded_ex_mfcc_list = []
    i =0
    a=int((duration-1)/0.5)
    for time in range(a) :
        y_split=split_wav(y,sr,i,i+1)
        mfcc = librosa.feature.mfcc(y_split, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)
        mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
        padded_mfcc = pad2d(mfcc, 100)
        np_padded_mfcc=np.array(padded_mfcc)
        np_padded_ex_mfcc=np.expand_dims(np_padded_mfcc, -1)
        np_padded_ex_mfcc_list.append(np_padded_ex_mfcc)
        i +=0.5

    word_list=np.argmax(model.predict(np.array(np_padded_ex_mfcc_list)),axis = 1)
    
    start_time = 0
    start_i = 0
    end_cnt = 0
    time_list2 = []
    for i in range(len(word_list)-1) :
        if word_list[i] == 1 :
            end_cnt += 1
            if (word_list[i+1] == 0) & (end_cnt >=2) :
                if start_i > 50 :
                    time_list2.append([start_time,(i+1)*0.5+0.05])
                    start_time =(i+1)*0.5
                    start_i = 0
        else :
            end_cnt = 0
        start_i +=1
    time_list2.append([time_list2[-1][1],math.floor(duration*100)/100])
    return time_list2

In [22]:
def create_stt_script(file_path,time_list2) :
    import speech_recognition as sr
    recognizer = sr.Recognizer()
    recognizer.energy_threshold = 300

    #데이터 경로 바꿔주기
    muli_audio = sr.AudioFile(file_path)

    #데이터 경로 바꿔주기
    audio_seg= AudioSegment.from_wav(file_path)
    total_in_ms= len(audio_seg)

    Text_list = []
    Text_list2 = []
    for start,end in tqdm(time_list2) : #tqdm(range (1,int(total_in_ms/1000/30)+1)) :
        with muli_audio as source:
            s_audio = recognizer.record(source,offset = start,duration=(end-start))
        Text_list.append(recognizer.recognize_google(audio_data=s_audio, language="ko-KR",show_all=True))
    last_list = []
    for i in Text_list :
        if i!= [] :
            last_list.append(i['alternative'][0]['transcript'])
        else :
            last_list.append("")
    df=pd.DataFrame(time_list2,columns=['start_time','end_time'])
    df["script"]=last_list
    df.to_csv(f"{file_name[:-4]}.csv",encoding="utf-8-sig",index=False)
    return df

In [23]:
def modify_script(data) :
    sentence_list = []
    for script in data["script"]: 
        sentence_list.append(kkm.sentences(script))
    script_list=[]
    new_sentence_list = []
    for sentences in sentence_list :
        script =[]
        for st in sentences :
            pos=kkm.pos(st)
            if (pos[-1][1] =='EFN') | (pos[-1][1] =='EFA'):
                script.append(st +". ")
            elif pos[-1][1] =='EFQ':  
                script.append(st +"? ")
            elif (pos[-1][1] =='EFI') | (pos[-1][1] =='EFO') :   
                script.append(st +"! ")
            else :
                script.append(st +" ")
        script_list.append(script)
    for i in range(len(script_list)-1) :
        if (script_list[i][-1][-2:] != '. ') & (script_list[i][-1][-2:] != '? ') & (script_list[i][-1][-2:] != '! ') :
            # 앞으로 밀착
            if len(script_list[i+1][0]) <=20 :
    #             data.iloc[i,1] = data.iloc[i,1] + wps*len(script_list[i+1][0])
    #             data.iloc[i+1,0] =data.iloc[i+1,0] + wps*len(script_list[i+1][0])
                data.iloc[i,1] = data.iloc[i,1] + ((data["end_time"]-data["start_time"])/len(data["script"]))[i]*len(script_list[i+1][0])
                data.iloc[i+1,0] =data.iloc[i+1,0] + ((data["end_time"]-data["start_time"])/len(data["script"]))[i]*len(script_list[i+1][0])
                script_list[i][-1]=script_list[i][-1] + script_list[i+1][0]
                script_list[i+1][0]=[] 
            # 뒤로 밀착
            elif len(script_list[i][-1]) <= 20:
    #             data.iloc[i,1] = data.iloc[i,1] - wps*len(script_list[i][-1])
    #             data.iloc[i+1,0] =data.iloc[i+1,0] - wps*len(script_list[i][-1])
                data.iloc[i,1] = data.iloc[i,1] - ((data["end_time"]-data["start_time"])/len(data["script"]))[i]*len(script_list[i][-1])
                data.iloc[i+1,0] =data.iloc[i+1,0] - ((data["end_time"]-data["start_time"])/len(data["script"]))[i]*len(script_list[i][-1])
                script_list[i+1][0]=script_list[i][-1] + script_list[i+1][0]
                script_list[i][-1]=[]
    final_script_list = []
    for script in script_list :
        sc_temp = ""
        for sc in script :
            if sc !=[] :
                sc_temp = sc_temp+ str(sc)
        final_script_list.append(sc_temp)
    data['final_script']=final_script_list
    return data

In [24]:
def extract_keywords(data) :
    text = ""
    for i in data["final_script"] :
        text += i + " "
          
    keyword_cnt = []
    textrank = TextRank(text)
    keywords=textrank.keywords(5) 
    for keyword in keywords :
        keyword_cnt.append(text.count(keyword))
    return keywords , keyword_cnt

In [25]:
def extract_summary(data) :
    text = ""
    for i in data["final_script"] :
        text += i + " "
    # 1. init
    mytokenizer: OktTokenizer = OktTokenizer()
    lexrank: LexRank = LexRank(mytokenizer)

    # 2. summarize (like, pre-computation)
    lexrank.summarize(text)

    # 3. probe (like, query-time)
    summaries: List[str] = lexrank.probe(0.05)
    summary_c = ""
    for summary in summaries:
        summary_c += summary + "\n"
    return summary_c

In [26]:

import pandas  as pd
# dsn=cx_Oracle.makedsn('222.102.104.172',1521,'XE')
# db=cx_Oracle.connect('hr','hr')
db = cx_Oracle.connect("hr/hr@222.102.104.172:1521/XE")
cursor=db.cursor()
cursor.execute("""select * from NOTE_CATEGORIES""")

row=cursor.fetchall()
colname=cursor.description
col=[]

for i in colname :
    col.append(i[0])
    
emp=pd.DataFrame(row,columns=col)
print(emp)

   NOTE_ID    USER_ID NOTE_NAME
0        1          a        국어
1        3          a        수학
2       52  feelwrite       회계학
3       49  feelwrite        국어
4       11          a       한국사
5        2          a       행정법
6        4          a       경제학
7       50  feelwrite       경제학


In [45]:
user_id = "a"
path_dir = 'data/공무원/'
file_name = "공무원_국어.mp4"
### mp4 -> mp3
# - moviepy.editor를 사용해 mp4파일을 mp3파일로 변환

video = VideoFileClip(os.path.join(path_dir+file_name))

video.audio.write_audiofile(os.path.join(path_dir+file_name[:-4]+".mp3"))

AudioSegment.from_mp3(path_dir+file_name[:-4]+".mp3").export(path_dir+file_name[:-4]+".wav", format="wav", bitrate="16k")
print(0)
duration = cal_duration(path_dir+file_name[:-4]+".wav")
print(1)

time_list2 = cut_time(path_dir+file_name[:-4]+".wav")
print(2)

data = create_stt_script(path_dir+file_name[:-4]+".wav",time_list2)
print(3)
data = modify_script(data)

print(4)

keywords,keyword_cnt=extract_keywords(data)
summary_c=extract_summary(data)

print(5)

insert_videos((user_id,1,file_name,duration,summary_c))

video_id = select_videos_seq()
for i in range(data.shape[0]) :    
    insert_video_analyses((user_id,video_id,round(data.iloc[i,0],1),round(data.iloc[i,1],1),data.iloc[i,3]))

for i in range(len(keywords)) : 
    insert_keywords((user_id,video_id,keywords[i],keyword_cnt[i]))
print("모두 완료")

MoviePy - Writing audio in data/공무원/공무원_국어.mp3


MoviePy - Done.
0
1


  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical i

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical i

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical i

...

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical i

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical issues were encountered "

  warnings.warn("Numerical i

2


100%|██████████████████████████████████████████████████████████████████████████████████| 52/52 [03:33<00:00,  4.10s/it]


3
4
5
모두 완료


In [28]:
pip install moviepy

Note: you may need to restart the kernel to use updated packages.


In [9]:
### mp4 -> mp3
# - moviepy.editor를 사용해 mp4파일을 mp3파일로 변환
path_dir = 'data/공무원/'
file_name = "공무원_국어.mp4"
print(path_dir + file_name)
print(path_dir+"new"+file_name[:-4]+".mp3")
print(os.path.exists(path_dir+"new"+file_name[:-4]+".mp3"))
#                                      경로변경해주기
video = VideoFileClip(os.path.join(path_dir+file_name))
#                                            새로 만들 파일 이름
# video.audio.write_audiofile(os.path.join(path_dir+"new"+file_name[:-4]+".mp3"))




### mp3 - > wav
# - from_mp3() : mp3파일이 있는경로  export() : 파일을 저장할 경로

#                         경로 변경해주기               새로 만들 파일 이름
# mp3=AudioSegment.from_mp3(path_dir+"new"+file_name[:-4]+".mp3")
video.export(path_dir+"new"+file_name[:-4]+".wav", format="wav", bitrate="16k")

data/공무원/공무원_국어.mp4
data/공무원/new공무원_국어.mp3
True


AttributeError: 'VideoFileClip' object has no attribute 'export'

In [7]:
video.audio.write_audiofile(os.path.join(path_dir+"new"+file_name[:-4]+".mp3"))

chunk:   0%|          | 0/32938 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in data/공무원/new공무원_국어.mp3


MoviePy - Done.


In [2]:
from os import path
from pydub import AudioSegment

# files                                                                         
src = "muli_sound.mp3"
dst = "test.wav"

# convert wav to mp3                                                            
sound = AudioSegment.from_mp3(src)
# sound.export(dst, format="wav")

In [3]:
pip install ffmpeg-python

  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=17cac7835c457a99ba0542752171b774f9a75bd41a1ec744f990b4c993356c85
  Stored in directory: c:\users\smhrd\appdata\local\pip\cache\wheels\6e\9c\ed\4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os


In [7]:
### mp4 -> mp3
# - moviepy.editor를 사용해 mp4파일을 mp3파일로 변환
path_dir = 'data/공무원/'
file_name = "공무원_국어.mp4"
print(path_dir + file_name)
print(path_dir+"new"+file_name[:-4]+".mp3")
print(os.path.exists(path_dir+"new"+file_name[:-4]+".mp3"))



video = VideoFileClip(os.path.join(path_dir+file_name))

video.audio.write_audiofile(os.path.join(path_dir+"new"+file_name[:-4]+".mp3"))

AudioSegment.from_mp3(path_dir+"new"+file_name[:-4]+".mp3").export(path_dir+"new"+file_name[:-4]+".wav", format="wav", bitrate="16k")

data/공무원/공무원_국어.mp4
data/공무원/new공무원_국어.mp3
True


In [8]:
mp3

<_io.BufferedRandom name='data/공무원/new공무원_국어.wav'>

In [20]:
import os
os.system("ffmpeg -i <audio> -i <video> -async 1 -c copy output.mp4")

1

In [5]:
import IPython


### mp4 -> mp3
# moviepy.editor를 사용해 mp4파일을 mp3파일로 변환

#                                      경로변경해주기
video = VideoFileClip(os.path.join("./data/muli.mp4"))
#                                            새로 만들 파일 이름
video.audio.write_audiofile(os.path.join("./data/muli_sound.mp3"))




### mp3 - > wav
# from_mp3() : mp3파일이 있는경로  export() : 파일을 저장할 경로


#                         경로 변경해주기               새로 만들 파일 이름
AudioSegment.from_mp3("data/muli_sound.mp3").export("data/muli_sound.wav", format="wav", bitrate="16k")

KeyboardInterrupt: 

In [3]:
!pip uninstall moviepy

^C


In [58]:
pip install ez_setup

  Created wheel for ez-setup: filename=ez_setup-0.9-py3-none-any.whl size=11013 sha256=40f4693780350b4c070a351d563a4b6622f0ace6d32557eb45381ce13edfac39
  Stored in directory: c:\users\smhrd\appdata\local\pip\cache\wheels\b1\68\a0\3224e9e4903eb884e775b63b1c72d82af5ac05a6288494adb8
Successfully built ez-setup
Note: you may need to restart the kernel to use updated packages.


In [29]:
def extract_keywords(data) :
    text = ""
    for i in data["new_word"] :
        text += i + " "
          
    keyword_cnt = []
    textrank = TextRank(text)
    keywords=textrank.keywords(5) 
    for keyword in keywords :
        keyword_cnt.append(text.count(keyword))
    return keywords , keyword_cnt

In [32]:
keywords,keyword_cnt=extract_keywords(data)
print(keywords)
print(keyword_cnt)

['생산', '시장', '이해', '자본주의', '가게']
[58, 34, 29, 21, 26]
